# Plotting CESM Output

Contents:
- Maps of CESM output (chemical compounds)
- Maps of CESM-SE output, and regional refinement projections
- Zonal average contour plots (including calculating ZA)
- Difference maps - how to choose different color bars
- Panel map plots with different legends
- Box-plots (percentiles)
- Polar graph (e.g. wind rose) [missing]
- Multi-panel graph (customizing sizes and locations of each panel)
- Add flow stream lines on contour map [missing]

---

## Let's read some CESM and CESM-SE data

To do this, we need to use `xarray`.

In [ ]:
import warnings
warnings.simplefilter("ignore") # Silence warnings
%matplotlib inline
import xarray as xr
import numpy as np

And now lets read CAM-chem on SE ne30 grid from GLADE.

In [ ]:
file_pattern = "/glade/scratch/emmons/PYTHON_TUTORIAL/f.e20.FCHIST.ne30_ne30_mg17.cam6_1_019.GEOS5_nudged.next_obs.timescale0.cam.h1.2013-*.nc"
ds = xr.open_mfdataset(file_pattern).load()

In [ ]:
ds

In [ ]:
# Set all invariant variables to be coords.
invariants = [v for v in ds.variables if not {'ncol', 'time'}.issubset(set(ds[v].dims))]
invariants.extend(['lat', 'lon', 'area'])
print(invariants)

In [ ]:
ds = ds.set_coords(invariants)
ds

# Visualizing unstructured data

Here we show how to visualize data on an unstructure grid using `cartopy` and `matplotlib`. To plot this dataset, we will need to read the topology of the dataset into a triangular mesh using `matplotlib`.

First, let's transform longitude values from 0 - 360 range to -180 - 180 range

In [ ]:
ds.lon.data.min(), ds.lon.data.max()

In [ ]:
lons = np.mod(ds.lon.data[0] - 180.0, 360.0) - 180.0
lons.min(), lons.max()

In [ ]:
lats = ds.lat.data[0]
print(lons.shape, lats.shape)

In [ ]:
import matplotlib.tri as tri
import matplotlib as mpl

In [ ]:
triang = tri.Triangulation(lons, lats)

Matplotlib just created an unstructured triangular grid consisting of npoints points and
ntri triangles. Since we didn't specify the indices of the three points that make
up the triangle, matpotlib automatically generates triangles using a [Delaunay triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation).

In [ ]:
triang.edges.shape

In [ ]:
triang.edges

Let's get one sample from the dataset to look at

In [ ]:
sample = ds.isel(ilev=slice(0,2), lev=0, time=slice(0, 1))[['O3']]
sample

In [ ]:
sample['O3'].data.flatten().shape

### Matplotlib & Cartopy

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
def make_quick_map():
    fig, ax = plt.subplots(figsize=(16, 9), subplot_kw=dict(projection=ccrs.PlateCarree()))
    ax.coastlines(resolution='50m')
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
make_quick_map()

In [ ]:
fig, ax = make_quick_map()
ax.tripcolor(triang, sample['O3'].values.flatten(), cmap='YlGn')
ax.coastlines()

Let's generalize our plotting utility function so that we can plot arbitrary data points

In [ ]:
def make_map(nrows, ncols, lat, lon):
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 9),
                             subplot_kw=dict(projection=ccrs.PlateCarree()), sharex=True, sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            axes[row][col].coastlines(resolution='50m')
            gl = axes[row][col].gridlines(draw_labels=True)
            gl.xlabels_top = gl.ylabels_right = False
            gl.xformatter = LONGITUDE_FORMATTER
            gl.yformatter = LATITUDE_FORMATTER
            axes[row][col].set_aspect('equal')       
    triang = tri.Triangulation(lon, lat)
    return fig, axes, triang

In [ ]:
def map_plotter(ds, data_var, nrows, ncols, lats, lons, lev_idx=0, cmap='gist_ncar'):
    sample = ds.isel(lev=lev_idx, time=slice(0, nrows*ncols))[[data_var]]
    fig, axes, triang = make_map(nrows, ncols, lats, lons)
    idxs = nrows * ncols
    for row in range(nrows):
        for col in range(ncols):
            idx = row * ncols + col
            tcp = axes[row][col].tripcolor(triang, sample[data_var].isel(time=idx).values.flatten(), cmap=cmap)
            axes[row][col].set_title(f'time={str(sample.time.values[idx])}')
    cax, kw = mpl.colorbar.make_axes([ax for ax in axes.flat])
    fig.suptitle(f'{sample[data_var].long_name} \n @ {sample.lev.long_name}={sample.lev.values} {sample.lev.units}', fontsize=20)
    fig.colorbar(tcp, cax=cax, label=f'units={sample[data_var].units}', **kw)

In [ ]:
# Plot Ozone concentration from 12 time steps, at lev with index=10
map_plotter(ds, 'O3', 4, 3, lats, lons, lev_idx=10)

In [ ]:
# Plot NO2 concentration from 4 time steps at lev with index=20
map_plotter(ds, 'NO2', 2, 2, lats, lons, lev_idx=20)

In [ ]:
map_plotter(ds, 'NO', 2, 2, lats, lons, lev_idx=30)

In [ ]:
map_plotter(ds, 'ISOP', 2, 2, lats, lons, lev_idx=20)

In [ ]:
map_plotter(ds, 'CLOUD', 2, 2, lats, lons, lev_idx=15)

In [ ]:
map_plotter(ds, 'T', 3, 3, lats, lons, lev_idx=8, cmap=mpl.cm.Spectral_r)

## HoloViews (hvplot) & GeoViews

### Exploring data of different dimensionality ranging from simple 1D data, to 2D image-like data, to multi-dimensional cubes of data

For these examples we’ll use the CO2 data from standard monthly average WACCM output:

In [ ]:
import hvplot.xarray
import geoviews as gv

In [ ]:
file_pattern = "/glade/scratch/emmons/PYTHON_TUTORIAL/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001.cam.h0.2013-*.nc"
data_vars = ['CO2']
ds1 = xr.open_mfdataset(file_pattern)
drop_vars = set(ds1.data_vars) - set(data_vars)
dset = ds1.drop(drop_vars).load()
co2 = dset.CO2
dset

#### 1D Plots 

Selecting the data at a particular lat/lon coordinate we get a 1D dataset of CO2 over time:

In [ ]:
co21d = co2.isel(lev=0, lat=10, lon=10)
co21d.hvplot()

Notice how the axes are already appropriately labeled, because xarray stores the metadata required. We can also further subselect the data and use * to overlay plots:

In [ ]:
co21d.hvplot(color='purple') * co21d.hvplot.scatter(marker='o', color='blue', size=15)

#### Selecting multiple 
If we select multiple coordinates along one axis and plot a chart type, the data will automatically be split by the coordinate:


In [ ]:
co2.isel(lev=0, lon=100, lat=[14, 20, 30, 60]).hvplot.line()

To plot a different relationship we can explicitly request to display the latitude along the y-axis and use the by keyword to color each longitude (or 'lon') differently:



In [ ]:
co2.isel(time=0, lev=0, lon=[0, 50, 100, 120, 130]).hvplot.line(y='lat', by='lon')

#### 2D Plots 
By default the `DataArray.hvplot()` method generates an image if the data is two-dimensional.


In [ ]:
co22d = co2.isel(time=5, lev=0)
co22d

In [ ]:
co22d.hvplot(width=600)

Alternatively we can also plot the same data using the `contour` and `contourf` methods, which provide a levels argument to control the number of iso-contours to draw:



In [ ]:
co22d.hvplot.contour(width=600, levels=20) + co22d.hvplot.contourf(width=600, levels=8)

#### n-D Plots 
However if the data has more than two dimensions it will default to a histogram without providing it further hints:



In [ ]:
co2.hvplot()

However we can tell it to apply a groupby along a particular dimension, allowing us to explore the data as images along that dimension with a slider:

In [ ]:
co2.hvplot(groupby=['time', 'lev'], width=600, cmap=mpl.cm.Spectral)

If we pick a different, lower dimensional plot type (such as a `line`) it will automatically apply a groupby over the remaining dimensions:

In [ ]:
co2.hvplot.line()

#### Statistical plots 

Statistical plots such as histograms, kernel-density estimates, or violin and box-whisker plots aggregate the data across one or more of the coordinate dimensions. For instance, plotting a KDE provides a summary of all the CO2 values but we can, once again, use the by keyword to view each selected latitude (or 'lat') separately:



In [ ]:
co2.sel(lev=[1, 2], lat=[25, 50, 75, 85], method='nearest').hvplot.kde('CO2', by=['lev', 'lat'], alpha=0.5)

In [ ]:
co2.sel(lev=[1, 2], lat=[25, 50, 75, 85], method='nearest').hvplot.hist('CO2', by=['lev', 'lat'], alpha=0.5)

In [ ]:
co2.sel(lev=[1, 2], lat=[25, 50, 75, 85], method='nearest').hvplot.box('CO2', by=['lev', 'lat'], box_alpha=0.5)

#### Geographic plots

To declare a geographic plot we have to supply a `cartopy.crs.CRS` (or coordinate reference system). Coordinate reference systems are described in the [GeoViews documentation](http://geoviews.org/user_guide/Projections.html) and the full list of available CRSs is in the [cartopy documentation](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html). 
Only certain hvPlot types support geographic coordinates, currently including: 

- `points`
- `polygons`
- `paths`
- `image`
- `quadmesh`
- `contour`
- `contourf`


In [ ]:
co2.isel(lev=[10, 15]).hvplot.quadmesh(
    'lon', 'lat', 'CO2', projection=ccrs.Orthographic(-90, 30),
    global_extent=True, width=600, height=540, cmap=mpl.cm.Spectral, 
    dynamic=True, rasterize=True
) * gv.feature.coastline


In [ ]:
co2.isel(lev=[10, 15]).hvplot.contourf(
    'lon', 'lat', 'CO2', crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    width=800, height=540, cmap=mpl.cm.Spectral, levels=8
) * gv.feature.coastline
